# Chapter 5. 오차역전파법
## 5.4 단순한 계층 구현하기
- 사과 쇼핑의 예를 파이썬으로 구현하려고 한다.
- 계산 그래프의 곱셈 노드 = MulLayer, 덧셈 노드 = AddLayer

### 5.4.1 곱셈 계층
- 모든 계층은 forward()와 backward() 공통의 메서드(인터페이스)를 가짐
    - forward()는 순전파, backward()는 역전파를 처리

In [1]:
# 곱셈 계층 구현
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):  # x와 y를 바꾼다.
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

- `__init__()`: 인스턴스 변수인 x와 y(순전파일 때 입력 값을 유지하기 위해서 사용) 초기화
- `forward()`: x와 y를 인수로 받아 두 값을 곱해 반환
- `backward()`: 상류에서 넘어온 미분(`dout`)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘림
- 사과 2개 구입하는 예제는 다음과 같이 나타낼 수 있다.

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 각 변수에 대한 미분-역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


- `backward()`의 호출 순서는 `forward()` 때와 반대
- `backward()`가 받는 인수 = 순전파의 출력에 대한 미분
    - ex. `mul_apple_layer`라는 곱셈 계층
        - 순전파 때는 `apple_price` 출력
        - 역전파 때는 `apple_priece`의 미분 값인 `dapple_price`를 인수로 받음
### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

- 초기화가 필요 없어 `__init__()`에서 `pass` 수행
- `forward()`: 입력받은 두 인수 x, y를 더해서 반환
- `backward()`: 상류에서 내려온 미분(`dout`)을 그대로 하류로 흘림

![image](https://user-images.githubusercontent.com/61455647/115655310-01641b80-a36e-11eb-9002-8ed307fdb48b.png)

- 덧셈 계층과 곱셈 계층으로 사과 2개와 귤 3개를 사는 경우를 나타내면 위와 같다.

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기
- 계산 그래프를 신경망에 적용하기
- 신경망을 구성하는 게층 각각을 클래스 하나로 구현
### 5.5.1 ReLU 계층

![image](https://user-images.githubusercontent.com/61455647/115656834-d929ec00-a370-11eb-89cf-47189656b027.png)

- ReLU의 수식과 x에 대한 y의 미분은 다음과 같다.
- 순전파 때 x가 0 이상이면, 역전파는 상류의 값을 그대로 하류로 흘리고, x가 0 이하면 역전파 때 하류로 신호를 보내지 않는다(= 0을 보낸다).

![image](https://user-images.githubusercontent.com/61455647/115657122-548b9d80-a371-11eb-9b1f-fd1d2b71f722.png)

- ReLU 계층은 계산 그래프로 다음과 같이 나타낼 수 있다.

In [6]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

- `mask`
    - True/False로 구성된 넘파이 배열
    - 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외에는 False로 유지

In [7]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


- 순전파 때 입력 값이 0 이하면 역전파 때의 값은 0이 되어야 한다.
- -> 역전파 때는 순전파 때 만든 mask를 써서 mask의 원소가 True인 곳에는 상류에서 전파된 dout = 0으로 설정
- ReLU 계층은 전기 회로의 스위치에 비유할 수 있다. 순전파 때 전류가 흐르고 있으면 스위치를 ON으로 하고, 흐르지 않으면 OFF로 한다. 역전파 때 스위치가 ON이면 전류가 그대로 흐르고, OFF면 흐르지 않는다.

### 5.5.2 Sigmoid 계층
- 시그모이드 함수: *y = 1 / (1 + exp(-x))*

![image](https://user-images.githubusercontent.com/61455647/115658859-9407b900-a374-11eb-8d73-c2ff546514f5.png)

- [×]와 [+] 노드 외에 [exp]와 [/] 노드가 새롭게 등장한다.
- 시그모이드 식의 계산은 국소적 계산의 전파로 이루어진다.
#### 1단계
- [/] 노드(*y = 1/x*)을 미분하면 다음 식이 된다.: *∂y/∂x = -(1/x^2) = -y^2*
- 역전파 때는 상류에서 흘러온 값에 *-y^2*을 곱해서 하류로 전달한다.

![image](https://user-images.githubusercontent.com/61455647/115661125-f7471a80-a377-11eb-83df-dbb287db9b96.png)

#### 2단계
- [+] 노드는 상류의 값을 하류로 내보낸다.

![image](https://user-images.githubusercontent.com/61455647/115661699-d3380900-a378-11eb-9c5a-7af29bccad37.png)

#### 3단계
- [exp] 노드는 *y = exp(x)* 연산을 수행하고, 미분은 다음과 같다.: *∂y/∂x = exp(x)*
- -> 계산 그래프에서 상류의 값에 순전파 때의 출력(= *exp(-x)*)을 곱해 하류로 전파한다.

![image](https://user-images.githubusercontent.com/61455647/115668754-74778d00-a382-11eb-874c-089ebf46a11b.png)

#### 4단계
- [×] 노드는 순전파 때의 값(= -1)을 서로 바꿔 곱한다.

![image](https://user-images.githubusercontent.com/61455647/115668997-c02a3680-a382-11eb-9313-e043a255b2fb.png)

- 역전파의 최종 출력인 *(∂L/∂y)y^2exp(-x)* 값이 하류 노드로 전파된다.
- 계산 그래프의 중간 과정을 묶어 하나의 [sigmoid] 노드 하나로 대체할 수 있다.

![image](https://user-images.githubusercontent.com/61455647/115671127-24e69080-a385-11eb-945d-e4b8129a0a9c.png)

- 간소화 버전은 역전파 과정의 중간 계산들을 생략할 수 있어 더 효율적인 계산이라고 할 수 있다.
- 노드를 그룹화하여 Sigmoid 계층의 세세한 내용을 노출하지 않고 입출력에만 집중할 수 있다.

![image](https://user-images.githubusercontent.com/61455647/115670143-0af87e00-a384-11eb-8697-020d6e4bfc48.png)

- *(∂L/∂y)y^2exp(-x)*는 위와 같이 나타낼 수 있다.
- => Sigmoid 계층의 역전파는 순전파의 출력 *y*만으로 계산할 수 있다.

![image](https://user-images.githubusercontent.com/61455647/115670591-8b1ee380-a384-11eb-86f3-6e367b3ebb3e.png)

In [8]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 5.6 Affine/Softmax 계층 구현하기
### 5.6.1 Affine 계층
- 신경망의 순전파에서 가중치 신호의 총합을 계산하기 위해 행렬의 곱(`np.dot()`) 사용

In [9]:
# Affine 계층 예제
X = np.random.rand(2)  # 입력
W = np.random.rand(2, 3)  # 가중치
B = np.random.rand(3)  # 편향

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B  # 뉴런의 가중치 합
print(Y)

(2,)
(2, 3)
(3,)
[0.98584264 0.98786044 1.20507235]


- **어파인 변환 affine transformation**: 신경망의 순전파 때 수행하는 행렬의 곱

![image](https://user-images.githubusercontent.com/61455647/115672771-d89c5000-a386-11eb-9f00-19075b18ef1f.png)

- 지금까지 계산 그래프는 노드 사이에 스칼라값이 흘렀지만, 이 예에서는 행렬이 흐른다.
- 행렬의 역전파를 전개하면 다음의 식을 얻을 수 있음

![image](https://user-images.githubusercontent.com/61455647/115679291-624f1c00-a38d-11eb-8f1b-7d892b69b370.png)

- W^T: 전치행렬
    - W의 (i, j) 위치의 원소를 (j, i) 위치로 바꾼 것
    - if) W의 형상이 (2, 3)이었다면, 전치행렬 W^T의 형상은 (3, 2)

![image](https://user-images.githubusercontent.com/61455647/115679488-99bdc880-a38d-11eb-99cd-51085849a793.png)

- 행렬을 이용한 계산 그래프의 역전파는 다음과 같다.

![image](https://user-images.githubusercontent.com/61455647/115680048-2c5e6780-a38e-11eb-910e-c651b30f1534.png)


- *X*와 *∂L/∂X*, *W*와 *∂L/∂W*은 같은 형상이다.

![image](https://user-images.githubusercontent.com/61455647/115680877-f077d200-a38e-11eb-9eff-3955f2a176fc.png)

- 행렬의 곱에서 대응하는 차원의 원소 수를 일치시켜야 하기 때문에 행렬의 형상에 주의해야 한다.

![image](https://user-images.githubusercontent.com/61455647/115681116-2e74f600-a38f-11eb-9a44-1146a80fcf45.png)

### 5.6.2 배치(batch)용 Affine 계층
- 데이터 N개를 묶어 순전파 하는 경우 계산 그래프는 다음과 같다.

![image](https://user-images.githubusercontent.com/61455647/115681435-7d229000-a38f-11eb-9111-49327d2bbfcc.png)

- X의 형상이 (N, 2)가 된 것이 기존과 다른 부분이다.

In [10]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)

print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


- 순전파의 편향 덧셈은 각각의 데이터에 더해진다. -> 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 함

In [11]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


- 데이터가 2개(N = 2)라고 가정한다. 편형의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구한다.(∵ `np.sum()`에서 `axis=0`의 총합을 구함)

In [12]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### 5.6.3 Softmax-with-Loss 계층

![image](https://user-images.githubusercontent.com/61455647/115691052-a4ca2600-a398-11eb-9433-cb0436b82a9d.png)

- 입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환되고, 마지막 Softmax 계층에 의해서 10개의 입력이 정규화(출력의 합이 1이 되도록 변형)된다.

![image](https://user-images.githubusercontent.com/61455647/115692299-d1cb0880-a399-11eb-96cb-62d4efc24923.png)

- Softmax-with-Loss 계층의 계산 그래프는 위와 같다. 이를 간소화하면 다음과 같다.

![image](https://user-images.githubusercontent.com/61455647/115693057-82390c80-a39a-11eb-98f3-782baaf1e763.png)

- 3클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)을 받는다.
- Softmax 계층: 입력 (a1, a2, a3) -(정규화)-> 출력 (y1, y2, y3)
- Cross Entropy Error 계층: 입력-Softmax의 출력 (y1, y2, y3) + 정답 레이블 (t1, t2, t3) -> 출력 손실 L
- **역전파의 결과**: Softmax 계층의 역전파 결과 = (y1-t1, y2-t2, y3-t3) = **Softmax 계층의 출력 - 정답 레이블의 오차**
    - ex. 정답 레이블이 (0, 1, 0)일 때 Softmax 계층이 (0.3, 0.2, 0.5) 출력 -> 역전파 (0.3, -0.8, 0.5)의 오차 전파
    - ex. 정답 레이블이 (0, 1, 0)일 때 Softmax 계층이 (0.01, 0.99, 0) 출력 -> 역전파 (0.01, -0.01, 0)의 오차 전파 -> 작은 오차 -> 학습하는 정도도 작음

In [13]:
from common.functions import softmax, cross_entropy_error

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## 5.7 오차역전파법 구현하기
### 5.7.1 신경망 학습의 전체 그림
- 신경망 학습의 순서
    1. 전제: 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다.
    2. 1단계-미니배치: 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 학습의 목표이다.
    3. 2단계-기울기 산출: 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.
    4. 3단계-매개변수 갱신: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.
    5. 4단계-반복: 1~3단계를 반복한다.
- 오차역전파법인 등장하는 단계는 '기울기 산출'
    - 4장에서 기울기를 구하기 위해서 수치 미분 사용
    - 수치 미분은 구현하기 쉽지만 계산이 오래 걸리기 때문에 오차역전파법을 이용해 기울기를 효율적이고 빠르게 구할 수 있다.
### 5.7.2 오차역전파법을 적용한 신경망 구현하기
- 2층 신경망을 TwoLayerNet 클래스로 구현하고자 한다.
    - Instance variables
        - `params`
            - 딕셔너리 변수로, 신경망의 매개변수 보관
            - `params['W1']`: 1번째 층의 가중치, `params['b1']`: 1번째 층의 편향
            - `params['W2']`: 2번째 층의 가중치, `params['b2']`: 2번째 층의 편향
        - `layers`
            - 순서가 있는 딕셔너리 변수로, 신경망의 계층 보관
            - `layers['Affine1']`, `layers['Relu1']`, `layers['Affine2']`와 같이 각 계층을 순서대로 유지
        - `lastLayer`
            - 신경망의 마지막 계층
            - 이 예에서는 SoftmaxWithLoss 계층
    - methods
        - `__init__(self, input_size, hidden_size, output_size, weight_init_std)`
            - 초기화 수행
            - 인수: 입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수, 가중치 초기화 시 정규분포의 스케일
        - `predict(self, x)`
            - 예측(추론) 수행
            - 인수 `x`: 이미지 데이터
        - `loss(self, x, t)`
            - 손실 함수의 값
            - 인수 `x`: 이미지 데이터, `t`: 정답 레이블
        - `accuracy(self, x, t)`: 정확도
        - `numerical_gradient(self, x, t)`: 가중치 매개변수의 기울기를 수치 미분 방식으로 구한다.
        - `gradient(self, x, t)`: 가중치 매개변수의 기울기를 오차역전파법으로 구한다.

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self. lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

- 신경망의 계층을 `OrderedDict`에 보관한다.
    - 순서가 있는 딕셔너리
    - = 딕셔너리에 추가한 순서를 기억한다.
    - -> 순전파 때 추가한 순서대로 각 계층의 `forward()`를 호출하기만 하면 됨.
    - -> 역전파 때 계층을 반대 순서로 호출하면 됨.
### 5.7.3 오차역전파법으로 구한 기울기 검증하기
- 기울기를 구하는 방법
    - 수치 미분
    - 해석적 방법 -> 오차역전파법: 매개변수가 많아도 효율적으로 계산할 수 있음.
- 수치 미분은 구현이 쉽고, 오차역전파법은 구현이 복잡하기 때문에 수치 미분의 결과와 오차역전파법의 결과를 비기ㅛ해 오차역전파법을 제대로 구현했는지 검증이 필요하다.
- -> **기울기 확인**: 두 방식으로 구한 기울기가 일치하는지를 확인

In [15]:
# 기울기 확인
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치 차이의 절대값을 구한 후, 그 절대값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ": " + str(diff))

W1: 5.318604853360182e-10
b1: 3.778228039993861e-09
W2: 5.190212139522985e-09
b2: 1.405525142508801e-07


- 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작다
- -> 오차역전파법으로 구한 기울기가 올바르다.
### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [16]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.10841666666666666 0.1078
0.90375 0.9084
0.9224166666666667 0.9241
0.9366833333333333 0.937
0.9453333333333334 0.9451
0.9515666666666667 0.9515
0.9565166666666667 0.9526
0.9602333333333334 0.9583
0.9599166666666666 0.9576
0.9672166666666666 0.963
0.9701333333333333 0.9642
0.9704 0.9637
0.9726333333333333 0.9661
0.9744 0.9685
0.9747333333333333 0.9688
0.9769333333333333 0.9674
0.9778 0.969
